<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train2_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import pandas as pd

In [114]:

train2 = pd.read_csv(r'/content/train_data2.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [115]:

train2['Target'] = train2['Target'].replace(['Others'],'Invalid')


In [116]:
train2['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [117]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [125]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train2_Xtfidf = tfidf_vect.fit_transform(train2['Sentence'])


In [119]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [135]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [136]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [137]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [138]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [139]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [126]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train2_Xtfidf,train2['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6692477876106194
                  precision    recall  f1-score   support

        Analysis       0.52      0.39      0.44       126
      Conclusion       0.69      0.56      0.62        45
           Facts       0.74      0.90      0.82       480
         Invalid       0.47      0.43      0.45       132
           Issue       0.51      0.43      0.47        51
Rule/Law/Holding       0.73      0.27      0.40        70

        accuracy                           0.67       904
       macro avg       0.61      0.50      0.53       904
    weighted avg       0.66      0.67      0.65       904



In [127]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [128]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 4, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9947365303485688, 0.9953505854588487, 0.999924667509294, 0.9805268473194878, 0.9986327537605316, 0.9967934308955387, 0.9971909527016516, 0.9979384503074725, 0.9933872157154067, 0.9815107257894884, 0.9847802597541521, 0.9990275227141573, 0.9950735439490053, 0.9984069770773913, 0.9891576483914623, 0.999440610247048, 0.9829746016261467, 0.9811033891610644, 0.9866291635016378, 0.9934883597179178, 0.991525292020041, 0.9877615725988393, 0.9992345506859154, 0.9996316613375197, 0.994400621221013, 0.9957919460021751, 0.995675168436558, 0.9960121432668481, 0.9910545438590319, 0.9962678887526281, 0.9977987132099431, 0.9993190286275458, 0.9966568040049114, 0.9878595743292473, 0.980290506687847, 0.9998712595187904, 0.9873746361552707, 0.982519154650512, 0.9927255486568802, 0.9953991481887534, 0.9955963788978596, 0.9899499826353771, 0.9992961391057368, 0.99651

In [129]:
unlabel_1=unlabel_1.loc[[1, 4, 5, 6, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 31, 33, 35, 37, 39, 40, 41, 45, 48, 49, 50, 51, 52, 55, 56, 60, 62, 68, 70, 75, 76, 77, 78, 79, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts
11,1604,police notified deceased found lying mrs moers...,Facts


In [130]:
frame_1 = [train2,unlabel_1]
train2_1 = pd.concat(frame_1)
len(train2_1)

3522

In [131]:
x_train_1 = tfidf_vect.transform(train2_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train2_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.675177304964539
                  precision    recall  f1-score   support

        Analysis       0.57      0.46      0.51        94
      Conclusion       0.68      0.50      0.58        38
           Facts       0.74      0.90      0.81       357
         Invalid       0.56      0.49      0.52       121
           Issue       0.64      0.41      0.50        44
Rule/Law/Holding       0.50      0.29      0.37        51

        accuracy                           0.68       705
       macro avg       0.61      0.51      0.55       705
    weighted avg       0.66      0.68      0.66       705



In [132]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2]
[0.9994591043102646, 0.9984093254503662, 0.9923361547907268, 0.9988604676427371, 0.9999708662124631, 0.9877146269156545, 0.9948002226524993, 0.9932940468799436, 0.99213712050506, 0.9982672399232825, 0.9819800887373463, 0.9991173064956071, 0.9997244337760087, 0.9993457806443354, 0.9980301307381515, 0.9873121022696006, 0.9985927801244334, 0.9854919756590363, 0.9885634611688758, 0.9997962903754422, 0.9835709123822611, 0.9997154805639472, 0.9998943932364796, 0.9993113874103533, 0.9988279918236296, 0.9989106877239498, 0.9828612176780793, 0.99629365673462, 0.9970905601473223, 0.9950560027869281, 0.9996459927882273, 0.9966850970097015, 0.9965640704283688, 0.9900640818835199, 0.9920617865847068]
[2, 4, 6, 8, 12, 15, 20, 21, 24, 29, 31, 36, 37, 39, 42, 44, 49, 50, 57, 59, 62, 63, 64, 69, 70, 71, 75, 83, 85, 86, 88, 90, 91, 95, 99]
35
35
35


In [140]:
unlabel_2 = unlabel_2.loc[[2, 4, 6, 8, 12, 15, 20, 21, 24, 29, 31, 36, 37, 39, 42, 44, 49, 50, 57, 59, 62, 63, 64, 69, 70, 71, 75, 83, 85, 86, 88, 90, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts
8,710,appellant indicted robbery indictment alleging...,Analysis
12,2085,mrs joyce wisnoski sister appellant testified ...,Facts


In [141]:
frame_2 = [train2_1,unlabel_2]
train2_2 = pd.concat(frame_2)
len(train2_2)

3557

In [142]:
x_train_2 = tfidf_vect.transform(train2_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train2_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6482830385015609
                  precision    recall  f1-score   support

        Analysis       0.42      0.44      0.43       142
      Conclusion       0.75      0.45      0.56        47
           Facts       0.73      0.90      0.81       501
         Invalid       0.48      0.40      0.43       149
           Issue       0.79      0.25      0.38        60
Rule/Law/Holding       0.62      0.26      0.36        62

        accuracy                           0.65       961
       macro avg       0.63      0.45      0.50       961
    weighted avg       0.64      0.65      0.63       961



In [143]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2]
[0.9939778198425894, 0.9892930916978224, 0.9911782043387478, 0.9986894839325766, 0.9990553038895643, 0.999317441007152, 0.9861100752042962, 0.9996839989679529, 0.9908635325441585, 0.9843151636859605, 0.9990923763806531, 0.9853723035344124, 0.9952491113297366, 0.9847687848785993, 0.9979955366570019, 0.9984028600449234, 0.993712224758774, 0.9973295527488562, 0.984046179874066, 0.9904431151851402, 0.9937223864201626, 0.991011062787742, 0.9880436749819087, 0.9876470019056303, 0.9843987224094574, 0.9854669572771964, 0.9984708819208035, 0.9998732073364606, 0.9851002776775647, 0.995102813013436, 0.9988656950570848, 0.9974127383460383, 0.9997396562179879, 0.9869827981196934, 0.997507083893371, 0.9842405526525455, 0.9890914070671118, 0.9995809066698965, 0.9926103819870833]
[0, 2, 3, 7, 8, 12, 19, 20, 23, 26, 27, 29, 30, 33, 36, 43, 46, 47, 48, 49, 51, 52, 53, 54, 56, 61, 63, 65,

In [144]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 7, 8, 12, 19, 20, 23, 26, 27, 29, 30, 33, 36, 43, 46, 47, 48, 49, 51, 52, 53, 54, 56, 61, 63, 65, 66, 67, 68, 70, 72, 73, 75, 79, 85, 89, 91],:]
frame_3 = [train2_2,unlabel_3]
train2_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train2_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train2_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6666666666666666
                  precision    recall  f1-score   support

        Analysis       0.50      0.45      0.47       101
      Conclusion       0.70      0.62      0.66        48
           Facts       0.74      0.88      0.81       368
         Invalid       0.49      0.42      0.45       115
           Issue       0.77      0.48      0.59        42
Rule/Law/Holding       0.48      0.26      0.34        46

        accuracy                           0.67       720
       macro avg       0.61      0.52      0.55       720
    weighted avg       0.65      0.67      0.65       720



In [145]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2]
[0.9995152605449227, 0.9819606315685864, 0.9909927156664744, 0.9971755449591079, 0.9911641325361887, 0.9991897847639741, 0.9982650310501511, 0.9867786291657021, 0.9855492704833404, 0.9995917526973461, 0.9959353860631884, 0.9978796613122545, 0.9898722556415629, 0.9987796195043877, 0.998899151929865, 0.998661191132482, 0.9963799481721028, 0.9957509859925343, 0.9935251082434702, 0.9995207842062899, 0.9982114037614294, 0.9968407957486818, 0.9997857112730936, 0.9928956561638081, 0.9878156374300748, 0.987126949087788, 0.9957273363254734, 0.9920437685848751, 0.9974615397146919, 0.9940105877230908, 0.9931049800303603, 0.9919421647341284, 0.9811355442302061, 0.9928699417056208, 0.9974945116457903, 0.9990423781743625, 0.9976774592598423, 0.9999596666086727, 0.9965014571608976, 0.997986686749851, 0.9901805837712043, 0.983373736024361, 0.9887573365255747, 0.989369

In [146]:
unlabel_4=unlabel_4.loc[[2, 4, 6, 7, 8, 10, 15, 16, 17, 18, 21, 22, 25, 27, 28, 29, 30, 31, 32, 34, 37, 38, 40, 42, 46, 49, 50, 55, 56, 57, 58, 60, 61, 64, 67, 70, 71, 75, 80, 85, 87, 94, 96, 97, 99 ],:]

In [147]:
frame_4 = [train2_3,unlabel_4]
train2_4 = pd.concat(frame_4)
len(train2_4)
x_train_4 = tfidf_vect.transform(train2_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train2_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.649390243902439
                  precision    recall  f1-score   support

        Analysis       0.52      0.40      0.45       153
      Conclusion       0.55      0.42      0.48        50
           Facts       0.73      0.89      0.81       503
         Invalid       0.48      0.39      0.43       161
           Issue       0.49      0.40      0.44        55
Rule/Law/Holding       0.57      0.37      0.45        62

        accuracy                           0.65       984
       macro avg       0.56      0.48      0.51       984
    weighted avg       0.63      0.65      0.63       984



In [148]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 3, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0]
[0.9835704866049845, 0.9986913237410033, 0.996710310675327, 0.9933960578147547, 0.98671526891422, 0.9859395773784677, 0.9973931169611265, 0.9829850493817166, 0.9972856472354256, 0.9993490236517071, 0.9874871814517967, 0.9863865759629611, 0.9968721575093502, 0.9997345226040366, 0.9853991245612229, 0.9933317904267871, 0.9994710884316547, 0.9962158906655093, 0.9914969856981842, 0.9992290141382286, 0.9976601549233941, 0.9994525706719124, 0.999482110875928, 0.9958230894680268, 0.9856995947274321]
[1, 3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 52, 53, 60, 65, 82, 86, 94, 95, 97, 103]
25
25
25


In [149]:
unlabel_5 = unlabel_5.loc[[1, 3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 52, 53, 60, 65, 82, 86, 94, 95, 97, 103],:]

In [151]:
frame_5 = [train2_4,unlabel_5]
train2_5 = pd.concat(frame_5)
len(train2_5)
x_train_5 = tfidf_vect.transform(train2_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train2_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6730245231607629
                  precision    recall  f1-score   support

        Analysis       0.45      0.44      0.44        95
      Conclusion       0.69      0.52      0.59        42
           Facts       0.78      0.87      0.82       382
         Invalid       0.48      0.45      0.47       108
           Issue       0.63      0.47      0.54        51
Rule/Law/Holding       0.59      0.41      0.48        56

        accuracy                           0.67       734
       macro avg       0.60      0.53      0.56       734
    weighted avg       0.66      0.67      0.66       734



In [152]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6717850287907869
